In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import os
os.chdir('drive/MyDrive/본인경로/이번주경로/')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, Lambda, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model

from plot_output import plot_latent_space, plot_denoising_results, gan_images

In [ ]:
# 데이터 로드 및 전처리
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') '''전처리 추가'''
x_test = x_test.astype('float32') '''전처리 추가'''

In [ ]:
x_train.shape, x_test.shape

In [ ]:
for idx in range(10):
  plt.imshow(x_train[idx], cmap='gray')
  plt.show()

# 1. Autoencoder

In [ ]:
# 입력 데이터 차원
input_dim = 28*28  # 28x28 픽셀 이미지가 1차원 벡터로 변환된 형태
encoding_dim = 32  # 잠재 공간의 차원 (숫자 바꾸면서 성능 비교해보셔도 됩니다)

In [ ]:
# 인코더 정의
encoder = Sequential([
    Flatten(input_shape=(28, 28)),  # 입력 이미지를 1차원 벡터로 변환
    '''추가'''  # 인코딩 레이어
])

# 디코더 정의
decoder = Sequential([
    '''추가'''  # 디코딩 레이어
    Reshape((28, 28))  # 1차원 벡터를 다시 28x28 이미지로 변환
])

# 오토인코더 모델 정의
autoencoder = Sequential([encoder, decoder])

# 손실 함수로 binary_crossentropy 사용
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# 모델 훈련
autoencoder.fit('''모델 입력, 출력 추가'''
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=('''모델 입력, 출력 추가'''))

In [ ]:
# 재구성 결과 확인
decoded_imgs = autoencoder.predict(x_test)

In [ ]:
n = 10  # 시각화할 이미지 개수
for i in range(n):
  # 원본 이미지
  plt.figure(figsize=(3,3))
  plt.title("Original")
  plt.imshow(x_test[i], cmap='gray')
  plt.show()
  # 재구성된 이미지
  plt.figure(figsize=(3,3))
  plt.title("Generated")
  plt.imshow(decoded_imgs[i], cmap='gray')
  plt.show()

# 2. VAE

In [ ]:
latent_dim = 2

# 인코더 정의
encoder = Sequential([
    Input(shape=(28, 28)),  # 입력 이미지 크기
    Flatten(),  # 입력 이미지를 1차원 벡터로 변환
    '''Dense 128'''  # 중간 레이어
    '''Dense 출력 2개'''  # z_mean과 z_log_var를 함께 출력
], name='encoder')
encoder.summary()


# 샘플링 레이어를 정의합니다
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = tf.split(inputs, num_or_size_splits=2, axis=1)
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# 디코더 정의
decoder = Sequential([
    Input(shape=(latent_dim,)),  # 잠재 공간 입력
    Dense(128, activation='relu'),  # 중간 레이어
    '''Dense 출력 이미지 크기 만큼''',  # 출력 레이어
    Reshape((28, 28))  # 1차원 벡터를 다시 28x28 이미지로 변환
], name='decoder')
decoder.summary()

# VAE 모델 정의
vae = Sequential([encoder, Sampling(), decoder], name='vae')

# 손실 함수 정의
def vae_loss(inputs, outputs, z_mean_log_var):
    z_mean, z_log_var = tf.split(z_mean_log_var, num_or_size_splits=2, axis=1)
    reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(inputs, outputs)) * 28 * 28
    kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
    return reconstruction_loss + kl_loss

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam()
vae.compile(optimizer, loss=lambda inputs, outputs: vae_loss(inputs, outputs, encoder(inputs)))
# 모델 학습
vae.fit('''모델 입력, 출력 추가''', epochs=30, batch_size=128, validation_data=('''모델 입력, 출력 추가'''))

In [ ]:
plot_latent_space(vae)

# 3. GAN

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)

In [ ]:
# 생성기 모델 정의
def build_generator():
    model = Sequential([
        '''생성기 입력은 노이즈'''
        '''생성기 출력은 이미지 크기 만큼'''
        Reshape((28, 28, 1))  # 784차원 벡터를 28x28 이미지로 변환
    ])
    return model

# 판별기 모델 정의
def build_discriminator():
    model = Sequential([
        '''판별기 입력은 이미지'''
        Dense(128, activation='relu', input_dim=784),
        '''판별기 출력은 출력이 1개'''
    ])
    return model

# GAN 모델 구축 및 학습
def build_gan(generator, discriminator):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False
    gan_input = tf.keras.Input(shape=(100,))
    gan_output = discriminator(generator(gan_input))
    gan = tf.keras.Model(gan_input, gan_output)
    gan.compile(optimizer='adam', loss='binary_crossentropy')
    return gan

# 모델 생성
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# 하이퍼파라미터 설정
batch_size = 128
epochs = 10000
sample_interval = 1000
noise_dim = 100



In [ ]:
# 학습 과정
def train(generator, discriminator, gan, x_train, batch_size, epochs, noise_dim, sample_interval):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # 판별기 훈련
        idx = np.random.randint(0, x_train.shape[0], half_batch)
        real_imgs = x_train[idx]

        noise = np.random.normal(0, 1, (half_batch, noise_dim))
        fake_imgs = generator.predict(noise)

        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        discriminator.train_on_batch(real_imgs, real_labels)
        discriminator.train_on_batch(fake_imgs, fake_labels)

        # 생성기 훈련
        noise = np.random.normal(0, 1, (batch_size, noise_dim))
        valid_y = np.ones((batch_size, 1))

        gan.train_on_batch(noise, valid_y)

        # 주기적으로 이미지 샘플링
        if epoch % sample_interval == 0:
            gan_images(generator, epoch, noise_dim)

In [ ]:
train(generator, discriminator, gan, x_train, batch_size, epochs, noise_dim, sample_interval)

# 4. Diffusion

In [ ]:
# 노이즈 추가 함수
def add_noise(data, noise_factor=0.5):
    noisy_data = data + noise_factor * np.random.normal(size=data.shape)
    noisy_data = np.clip(noisy_data, 0., 1.)
    return noisy_data

# 노이즈 제거 모델 정의
def build_denoising_model():
    model = tf.keras.Sequential([
        Input(shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), padding='same'),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (3, 3), activation='sigmoid', padding='same')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model


# 노이즈 데이터 생성
noise_factor = 0.5
x_train_noisy = '''학습 데이터 노이즈 입히기'''
x_test_noisy = '''테스트 데이터 노이즈 입히기'''

# 모델 학습
denoising_model = build_denoising_model()
denoising_model.fit('''입력과 출력은 노이즈 추가된 입력, 노이즈 없는 출력''', epochs=10, batch_size=128, validation_data=('''입력과 출력은 노이즈 추가된 입력, 노이즈 없는 출력'''))

# 노이즈 제거 결과 시각화
plot_denoising_results(denoising_model, x_test_noisy, x_test)

# +추가 실습!

In [ ]:
!pip install diffusers transformers scipy

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt

# 모델 및 토크나이저 로드
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipeline = StableDiffusionPipeline.from_pretrained(model_id)
pipeline = pipeline.to(device)



In [ ]:
# 텍스트 프롬프트를 사용하여 이미지 생성
prompt = "A scenic landscape with mountains and a lake"
with torch.autocast("cuda"):
    image = pipeline(prompt)[0]

# 이미지 시각화
plt.imshow(image[0])
plt.axis("off")
plt.show()

In [ ]:
# 텍스트 프롬프트를 사용하여 이미지 생성
prompt = "A monkey in a swimming pool"
with torch.autocast("cuda"):
    image = pipeline(prompt)[0]

# 이미지 시각화
plt.imshow(image[0])
plt.axis("off")
plt.show()